In [92]:
bearer_token="AAAAAAAAAAAAAAAAAAAAAAT4iAEAAAAA5OcF%2BJbRdQJqcT9w70TGCsHA%2B0c%3DJs1sHTVR8KNtC4caswoarwdO2ntDuAEHROTRQhxilbIfaLTxmG"
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r

In [93]:
import requests
import pandas as pd
import time
import numpy as np

In [94]:
# Number of requests allowed per time window
RATE_LIMIT = 15
# Time window (in seconds)
TIME_WINDOW = 15

# Bucket for tracking remaining requests
bucket = RATE_LIMIT
# Timestamp of last bucket refill
last_refill = time.time()

def connect_to_endpoint(url, params):
    global bucket
    global last_refill
    # Check if bucket is empty
    if bucket <= 0:
        # Check if time window has expired
        current_time = time.time()
        if current_time - last_refill > TIME_WINDOW:
            # Refill bucket and update timestamp
            bucket = RATE_LIMIT
            last_refill = current_time
        else:
            # Wait for time window to expire
            time_to_wait = last_refill + TIME_WINDOW - current_time
            time.sleep(time_to_wait)
            bucket = RATE_LIMIT
    # Send request
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    # Decrement bucket
    bucket -= 1
    return response

In [95]:
def get_followers(user_id):

    # Get list of followers
    followers = []
    # for page in tweepy.Cursor(api.get_followers, screen_name=username).pages():
    #     followers.extend(page)
    search_url = "https://api.twitter.com/2/users/{}/followers".format(user_id)
    query_params={}
    followers = connect_to_endpoint(search_url, query_params)
    follower=[]
    for fol in (followers.json()['data'][0:100]):
        follower.append(fol['id'])
    return follower

In [96]:
def get_recent_tweets_community(user_id):
    # Connect to the Twitter API using your API keys
        
    # Get the recent tweets from the specified user
    search_url='https://api.twitter.com/2/users/{}/tweets?max_results=10'.format(user_id)
    query_params={}
    tweets = connect_to_endpoint(search_url, query_params)    
    tweet=[]
    if "data" in tweets.json():
        for tw in tweets.json()['data']:
            tweet.append(tw["text"])
    return tweet

In [97]:
def get_recent_tweets(user_id):
    # Connect to the Twitter API using your API keys
        
    # Get the recent tweets from the specified user
    search_url='https://api.twitter.com/2/users/{}/tweets?max_results=10'.format(user_id)
    query_params={}
    tweets = connect_to_endpoint(search_url, query_params)    
    tweet=[]
    if "data" in tweets.json():
        for tw in tweets.json()['data']:
            tweet.append(tw["text"])
    return tweet

In [98]:
fwords = dict(
    auxiliary_verbs=['am',
                    'had',
                    'could',
                     'is',
                    'did',
                    'does',
                    'are',
                    'shall',
                    'do',
                    'was',
                    'will',
                    'need',
                    'were',
                    'should',
                    'ought to',
                    'being',
                    'would',
                    'dare',
                    'been',
                    'may',
                    'going to',
                    'be',
                    'might',
                    "be able to",
                    "has",
                    "must",
                    "have to",
                    "have",
                    "can",
                    "had better"],
    articles=['a',
              'an',
              'the'],
    
    common_adverbs=
        ['accidentally',
         'afterwards',
         'almost',
         'always',
         'angrily',
         'annually',
         'anxiously',
         'awkwardly',
         'badly',
         'blindly',
         'boastfully',
         'boldly',
         'bravely',
         'briefly',
         'brightly',
         'crossly',
         'cruelly',
         'daily',
         'defiantly',
         'deliberately',
         'doubtfully',
         'easily',
         'elegantly',
         'enormously',
         'enthusiastically',
         'equally',
         'even',
         'eventually',
         'exactly',
         'faithfully',
         'gladly',
         'gracefully',
         'greedily',
         'happily',
         'hastily',
         'honestly',
         'hourly',
         'hungrily',
         'innocently',
         'inquisitively',
         'irritably',
         'joyously',
         'justly',
         'kindly',
         'lazily',
         'nearly',
         'neatly',
         'nervously',
         'never',
         'noisily',
         'not',
         'obediently',
         'obnoxiously',
         'often',
         'only',
         'painfully',
         'perfectly',
         'politely',
         'poorly',
         'powerfully',
         'reluctantly',
         'repeatedly',
         'rightfully',
         'roughly',
         'rudely',
         'sadly',
         'safely',
         'seldom',
        'selfishly',
        'seriously',
        'shakily',
        'sharply',
        'shrilly',
        'shyly',
        'silently',
        'sternly',
        'successfully',
        'suddenly',
        'suspiciously',
        'swiftly',
        'tenderly',
        'tensely',
        'thoughtfully',
        'tightly',
        'tomorrow',
        'too',
        'truthfully',
        'unexpectedly',
        'very',
        'victoriously'],
    
    personal_pronouns=["i",
                       "you",
                       "he",
                       "she",
                       "it",
                       "we",
                       "they",
                       "them",
                       "us", 
                       "him", 
                       "her", 
                       "his", 
                       "hers", 
                       "its", 
                       "theirs", 
                       "our", 
                       "your"],
    
    impersonal_pronouns=["one",
                        "they",
                        "it",
                        "you"],
    
    prepositions=['about',
                  'above',
                  'across',
                  'after',
                  'ago',
                  'at',
                  'below',
                  'by',
                  'down',
                  'during',
                  'for',
                  'from',
                  'in',
                  'into',
                  'off',
                  'on',
                  'over',
                  'past',
                  'since',
                  'through',
                  'to',
                  'under',
                  'until',
                  'up',
                  'with'],
    
    negations=['doesn’t',
               'isn’t',
               'wasn’t',
               'shouldn’t',
               'wouldn’t',
               'couldn’t',
               'won’t',
               'can’t',
               'don’t',
               'hardly',
               'scarcely',
               'barely',
               'no',
               'not',
               'none',
               'no one',
               'nobody',
               'nothing',
               'neither',
               'nowhere',
               'never'],
    conjunctions=['after',
                  'after all',
                  'although',
                  'and',
                  'as', 
                  'as if',
                  "as long as",
                  "as much as",
                  "as soon as",
                  "as though",
                  "barely when",
                  "because","Before",
                  "but",
                  "by the time",
                  "bonsequently",
                  "bum",
                  "either or",
                  "even",
                  "even if",
                  "even though",
                  "ever since",
                  "every time",
                  "finally",
                  "for",
                  "furthermore",
                  "hardly when",
                  "hence",
                  "how",
                  "however",
                  "if",
                  'if … then',
                  'if only',
                  'if then',
                  'if when',
                  'in addition',
                  'in order that',
                  'inasmuch',
                  'incidentally',
                  'just as',
                  'lest',
                  'likewise',
                  'meanwhile',
                  'nor',
                  'now',
                  'now since',
                  'now that',
                  'now when',
                  'once',
                  'only if',
                  'or',
                  'or else',
                  'otherwise',
                  'provided',
                  'provided that',
                  'rather than',
                  'scarcely when',
                  'since'],
    
    quantifiers=['much',
                 'a bit',
                 'little',
                 'great deal of',
                 'large quantity of',
                 'a little',
                 'very little',
                 'a large amount of',
                 'a majority of',
                 'a great number of',
                 'several',
                 'many',
                 'a large number of',
                 'a number of',
                 'few',
                 'a few',
                 'very few',
                 'all',
                 'enough',
                 'none',
                 'no',
                 'some',
                 'more',
                 'most',
                 'lots of',
                 'less',
                 'least',
                 'any',
                 'not any',
                 'plenty of']
)
 
def calculateLSM(user):
    user_newom=user
    followers=get_followers(user_newom)
    tweets_followers=[]
    for i in followers:

        tweets_followers.append(get_recent_tweets_community(i))
        
    '''
    FWC function for the user that triggred the Newom
    '''

    df= get_recent_tweets(user_newom)

    #case 0 nombres de mots dans le dic 
    #case 1 nombres totals de mots dans les tweets 

    tableautot=[0,0] 

    tableaucategories=np.zeros(9)

    for i in range(len(df)):
        
        split = df[i].split()


        tableau=[0,0,0,0,0,0,0,0,0]

        for i in range(len(split)):
            cptcategories=0
            for j in (fwords.items()):

                if split[i].lower() in j[1]:

                    #print(split[i])
                    tableau[cptcategories]+=1

                cptcategories+=1
        tableautot[0]+=sum(tableau)
        tableautot[1]+=len(split)
        
        tableaucategories+=np.array(tableau)
        
    #print(tableautot)

    if tableautot[1]==0:
        FWC=0
        somme1=0
    else:
        FWC= tableautot[0]/tableautot[1]
        somme1= tableaucategories/tableautot[1]
        #print(tableaucategories)
    print("FWC" , FWC)
    print(somme1)


    '''
    FWC function for the BrandCommunity
    '''

    #FWC function 

    df2= tweets_followers

    #case 0 nombres de mots dans le dic 
    #case 1 nombres totals de mots dans les tweets 

    tableautot=[0,0] 

    tableaucategories=np.zeros(9)

    for i in range(len(df2)):
        
        if type(df2[i])== str:
            
        
            split = df2[i].split()


            tableau=[0,0,0,0,0,0,0,0,0]

            for i in range(len(split)):
                cptcategories=0
                for j in (fwords.items()):

                    if split[i].lower() in j[1]:

                        #print(split[i])
                        tableau[cptcategories]+=1

                    cptcategories+=1
            tableautot[0]+=sum(tableau)
            tableautot[1]+=len(split)

            tableaucategories+=np.array(tableau)

        #print(tableautot)
    if tableautot[1]!=0:
        FWC2= tableautot[0]/tableautot[1]
        somme2= tableaucategories/tableautot[1]

    else:
        FWC2=0
        somme2=0

    print("WC2" ,FWC2)
    print(somme2)


    '''
    LSM formula
    '''

    #LSM
    LSM= 1-(abs(FWC-FWC2/abs(FWC+FWC2+0.0001)))
    print("LSM :" ,LSM)
    return LSM

In [99]:
lsmscore = calculateLSM("809532994444591104")
#  musk :0.685483870967742 
# 809532994444591104 : 0.7124352331606217

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
FWC 0.329004329004329
[0.05194805 0.0952381  0.         0.01298701 0.004329   0.0952381
 0.004329   0.05627706 0.00865801]
WC2 0
0
LSM : 0.670995670995671


In [100]:
print(lsmscore)#.710411198600175

0.670995670995671
